# Time Series Project (NAME TBD)

---

**Project & Final Report Created By:** Mathias Boissevain & Rachel Robbins-Mayhill, April 22, 2022

---

In [1]:
# Import for data manipulation
import pandas as pd
import numpy as np

# Import for data viz
import seaborn as sns
from matplotlib import pyplot as plt

# Import for acquisition
import env
import os
import mathias_wrangle as mw

# Add for setting to see all rows and columns
pd.options.display.max_rows = None
pd.options.display.max_columns = None

# Import to ignore warnings
import warnings
warnings.filterwarnings('ignore')

## PROJECT DESCRIPTION:

## PROJECT GOAL: 

## INITIAL QUESTIONS: 

---

## I. ACQUIRE

10 POINTS -> ACQUISITION & PREPARATION
- Is data from the relevant tables included? 
- Is the data wrangling easily reproducible? 
- Can I import a function from an acquire/prep/wrangle module and have it give me the prepared data? 
- Are the steps taken for data acquisition and preparation well documented in the report notebook?

5 POINTS -> CODE STYLE
- Is the code well commented / documented? 
- Do functions and variables have descriptive names? 
- Is code broken up into functions / modules appropriately? - Is the code formatting consistent?

5 POINTS -> GENERAL STYLE
- Does your notebook have a good title? 
- Does the readme provide a description of the project and instructions on running your code? 
- Are headings used to organize the notebook? 
- Is the text free from grammatical and spelling errors?

In [2]:
ssdb = mw.prep_superstore(mw.get_superstore())

In [3]:
# Inspect df
ssdb.head()

,unnamed:_0,order_id,ship_date,ship_mode,segment,country,city,state,postal_code,sales,quantity,discount,profit,category,sub-category,customer_name,product_name,region_name,days_to_ship
order_date,,,,,,,,,,,,,,,,,,,
2014-01-04,977,CA-2014-112326,2014-01-08,Standard Class,Home Office,United States,Naperville,Illinois,60540.0,11.784,3.0,0.2,4.2717,Office Supplies,Labels,Phillina Ober,Avery 508,Central,4
2014-01-04,978,CA-2014-112326,2014-01-08,Standard Class,Home Office,United States,Naperville,Illinois,60540.0,272.736,3.0,0.2,-64.7748,Office Supplies,Storage,Phillina Ober,SAFCO Boltless Steel Shelving,Central,4
2014-01-04,979,CA-2014-112326,2014-01-08,Standard Class,Home Office,United States,Naperville,Illinois,60540.0,3.540,2.0,0.8,-5.4870,Office Supplies,Binders,Phillina Ober,GBC Standard Plastic Binding Systems Combs,Central,4
2014-01-09,942,CA-2014-135405,2014-01-13,Standard Class,Consumer,United States,Laredo,Texas,78041.0,31.200,3.0,0.2,9.7500,Technology,Accessories,Melanie Seite,Memorex Micro Travel Drive 8 GB,Central,4
2014-01-09,941,CA-2014-135405,2014-01-13,Standard Class,Consumer,United States,Laredo,Texas,78041.0,9.344,2.0,0.2,1.1680,Office Supplies,Art,Melanie Seite,Newell 312,Central,4


---

## II. PREPARE

---

### PREPARE - SPLIT the DATA

---

## III. EXPLORE

10 POINTS
- Are figures well labeled (title, x + y labels)? 
- Does the type of visualization make sense for the variables being explored? 
- Is color used in an appropriate way? 
- Are takeaways documented? 
- Are questions asked and answered? 
- Are statistical tests used appropriately to back up conclusions?

#### Univariate:

#### Bivariate:

#### Multivariate:

---

### EXPLORE - Questions

#### Question 1:

##### Visualization:

##### Stats Testing:

#### Answer: 

---

#### Question 2:

##### Visualization:

##### Stats Testing:

#### Answer: 

---

#### Question 3:

##### Visualization:

##### Stats Testing:

#### Answer: 

---

#### Question 4:

##### Visualization:

##### Stats Testing:

#### Answer: 

---

### EXPLORATION TAKEAWAYS

---

## MODELING (OPTIONAL)

---

# SUMMARY

---

## RECOMMENDATIONS

10 POINTS -> RECOMMENDATIONS
- Are recommendations that answer the original business question present, 
- clearly communicated, 
- backed by data, 
- and supported by visualizations?

---

## NEXT STEPS